In [1]:
import csv
import numpy as np
import collections

In [2]:
data = []
with open('plays_clean.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data.append(row)

In [3]:
grouped_by_game = collections.defaultdict(list)
for line in data:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

In [4]:
for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass

In [79]:
def put_into_bucket(play_dict):
    yard = play_dict['ydline'] // 10
    down = play_dict['down']
    if play_dict['togo'] <= 2:
        togo = 0
    elif play_dict['togo'] <= 4:
        togo = 1
    elif play_dict['togo'] <= 6:
        togo = 2
    elif play_dict['togo'] <= 8:
        togo = 3
    elif play_dict['togo'] <= 10:
        togo = 4
    elif play_dict['togo'] <= 15:
        togo = 5
    elif play_dict['togo'] <= 20:
        togo = 6
    else:
        togo = 7
    if play_dict['off'] < play_dict['def']:
        posession = 1
    else:
        posession = 0
    return (yard, down, togo, posession)

In [6]:
grouped_by_game[0][0]

{'def': 'Lions',
 'defscore': 0,
 'description': '(15:00) M.Tuiasosopo pass to J.Rice to OAK 28 for 8 yards (D.Evans).',
 'down': 1,
 'gameid': '20031102_OAK@DET',
 'off': 'Raiders',
 'offscore': 0,
 'time': 3600,
 'togo': 10,
 'ydline': 80}

In [7]:
game = grouped_by_game[0]
scores = collections.defaultdict(list)
for play in game:
    scores[play['off']].append(play['offscore'])
    scores[play['def']].append(play['defscore'])
    
for i in range(1, len(game)):
    for team in scores:
        if scores[team][i] != scores[team][i -1]:
            print(i, game[i])

6 {'down': '', 'defscore': 0, 'ydline': 35, 'off': 'Lions', 'offscore': 3, 'gameid': '20031102_OAK@DET', 'time': 3479, 'def': 'Raiders', 'togo': '', 'description': 'J.Hanson kicks 70 yards from DET 30 to end zone Touchback.'}
33 {'down': '', 'defscore': 0, 'ydline': 2, 'off': 'Lions', 'offscore': 10, 'gameid': '20031102_OAK@DET', 'time': 2762, 'def': 'Raiders', 'togo': '', 'description': 'J.Hanson kicks 63 yards from DET 30 to OAK 7. R.Jenkins to OAK 24 for 17 yards (A.Cobourne W.Rainer).'}
79 {'down': 1, 'defscore': 3, 'ydline': 66, 'off': 'Lions', 'offscore': 10, 'gameid': '20031102_OAK@DET', 'time': 1793, 'def': 'Raiders', 'togo': 10, 'description': '(14:53) O.Gary left tackle to DET 34 for no gain (T.Smith N.Harris).'}
87 {'down': '', 'defscore': 3, 'ydline': 33, 'off': 'Lions', 'offscore': 17, 'gameid': '20031102_OAK@DET', 'time': 1573, 'def': 'Raiders', 'togo': '', 'description': 'J.Hanson kicks 61 yards from DET 30 to OAK 9. R.Jenkins to OAK 21 for 12 yards (W.Rainer).'}
104 {'d

In [21]:
counter = 0
features = ['down', 'togo']
for game in grouped_by_game:
    for i, play in enumerate(game):
        missing = False
        for feature in features:
            if not play[feature]:
                missing = True
        if missing:
            counter += 1
            scores = collections.defaultdict(set)
            num_gaps = 1
            for j in range(i-1, i + 1):
                if 0 <= j < len(game):
                    scores[game[j]['off']].add(game[j]['offscore'])
                    scores[game[j]['def']].add(game[j]['defscore'])
                    for feature in features:
                        if not game[j][feature]:
                            num_gaps += 1
            score_changed = False
            for team in scores:
                if len(scores) >= 1:
                    score_changed = True
                #if max(scores[team]) - min(scores[team]) in (2,3):
                #    num_gaps -= 1
                #if max(scores[team]) - min(scores[team]) in (6,7,8):
                #    num_gaps -= 2
            #if num_gaps > 0:
            #    #print(play)
            #    pass
            if not score_changed:
                print(play)

In [97]:
def group_by_score_change(game, original_scores = None):
    if original_scores is None:
        original_scores = collections.defaultdict(int)
    changed_team = None
    changed_score = None
    for i, play in enumerate(game):
        if original_scores[play['off']] != play['offscore']:
            changed_team = play['off']
            #changed_score = play['offscore'] 
            changed_score = play['offscore'] - original_scores[changed_team]
            break
        if original_scores[play['def']] != play['defscore']:
            changed_team = play['def']
            #changed_score = play['defscore'] 
            changed_score = play['defscore'] - original_scores[changed_team]
            break     
    
    if changed_team is None:
        return [('', 0,game)]
    
    new_scores = collections.defaultdict(int)
    new_scores[play['off']] = play['offscore']
    new_scores[play['def']] = play['defscore']
    return [(changed_team, changed_score, game[:i+1])] + group_by_score_change(game[i+1:], new_scores)
    
    
    '''if changed_score in (2,3):
        return [game[:i+1]] + group_by_score_change(game[i+1:, new_scores])
    
    if changed_score == 6:
        j = i + 1
        if j < len(game):'''
            
    

In [53]:
def bucketable(play):
    if not play['down']:
        return False
    if play['down'] < 1:
        return False
    if play['down'] > 4:
        return False
    if not play['togo']:
        return False
    if play['togo'] < 1:
        return False
    if not play['ydline']:
        return False
    if play['ydline'] < 1:
        return False
    if not play['off'] or not play['def']:
        return False
    return True
    


In [119]:
def possession(play):
    if play_dict['off'] < play_dict['def']:
        return 1
    else:
        return 0

def bucket_group(group):
    team, diff, group = group
    answer = []
    if diff != 0:
        group = group[:-1]
    for i, play in enumerate(group):
        if play['time'] in (1800, 3600):
            if i == 0 or group[i-1]['time'] in (1800, 3600):
                answer.append([6])
        if bucketable(play):
            answer.append(put_into_bucket(play))
        else:
            answer.append([6])
    if team:
        pos = 1- sorted((play['off'], play['def'])).index(team)
        if answer[-1] == [6]:
            answer = answer[:-1]
        if diff == 2:
            answer.append([1, pos])
            answer.append([5, pos])
        elif diff == 3:
            answer.append([0, pos])
            answer.append([5, pos])
        elif diff == 6:
            answer.append([2, pos])
            answer.append([5, pos])
        elif diff == 7:
            answer.append([2, pos])
            answer.append([3, pos])
            answer.append([5, pos])
        elif diff == 8:
            answer.append([2, pos])
            answer.append([4, pos])
            answer.append([5, pos])
        elif diff != 0:
            answer.append([6])
    return answer
        

In [84]:
def bucket_game(game):
    grouped = group_by_score_change(game)
    answer = []
    for group in grouped:
        answer += bucket_group(group)
    return answer
        


In [113]:
group_by_score_change(grouped_by_game[0])[8]

('',
 0,
 [{'def': 'Lions',
   'defscore': 23,
   'description': '(5:45) R.Mirer pass intended for J.Porter INTERCEPTED by Bri.Walker at DET 43. Bri.Walker to OAK 14 for 43 yards (C.Garner).',
   'down': 1,
   'gameid': '20031102_OAK@DET',
   'off': 'Raiders',
   'offscore': 13,
   'time': 345,
   'togo': 10,
   'ydline': 70},
  {'def': 'Raiders',
   'defscore': 13,
   'description': '(5:41) O.Gary right end to DET 45 for 3 yards (C.Cooper).',
   'down': 1,
   'gameid': '20031102_OAK@DET',
   'off': 'Lions',
   'offscore': 23,
   'time': 341,
   'togo': 10,
   'ydline': 58},
  {'def': 'Raiders',
   'defscore': 13,
   'description': '(4:59) S.Bryson up the middle to 50 for 5 yards (T.Brayton).',
   'down': 2,
   'gameid': '20031102_OAK@DET',
   'off': 'Lions',
   'offscore': 23,
   'time': 299,
   'togo': 7,
   'ydline': 55},
  {'def': 'Raiders',
   'defscore': 13,
   'description': '(4:16) S.Bryson left tackle to OAK 45 for 5 yards (E.Barton).',
   'down': 3,
   'gameid': '20031102_OAK

In [120]:
bucket_game(grouped_by_game[1])

[(8, 1, 4, 1),
 (8, 2, 3, 1),
 (7, 3, 1, 1),
 (6, 1, 4, 1),
 (6, 2, 4, 1),
 (5, 1, 4, 1),
 (5, 2, 3, 1),
 (4, 3, 0, 1),
 (4, 4, 0, 1),
 (8, 1, 4, 0),
 (8, 2, 4, 0),
 (8, 3, 3, 0),
 (7, 4, 0, 0),
 (8, 4, 2, 0),
 (5, 1, 4, 1),
 (1, 1, 4, 1),
 (1, 2, 2, 1),
 (1, 3, 0, 1),
 (1, 4, 0, 1),
 [0, 1],
 [5, 1],
 (6, 1, 4, 0),
 (6, 2, 4, 0),
 (6, 3, 4, 0),
 (6, 4, 3, 0),
 (6, 1, 4, 1),
 (5, 1, 4, 1),
 (5, 2, 4, 1),
 (4, 3, 0, 1),
 (4, 1, 4, 1),
 (3, 2, 1, 1),
 (3, 3, 0, 1),
 (3, 4, 0, 1),
 (2, 1, 4, 1),
 (2, 2, 4, 1),
 (2, 2, 2, 1),
 (2, 3, 2, 1),
 (2, 4, 2, 1),
 [0, 1],
 [5, 1],
 (6, 1, 4, 0),
 (6, 2, 2, 0),
 (6, 3, 2, 0),
 (6, 4, 2, 0),
 (9, 1, 4, 1),
 (8, 2, 1, 1),
 (8, 3, 0, 1),
 (7, 1, 4, 1),
 (7, 2, 4, 1),
 (8, 2, 5, 1),
 (7, 3, 1, 1),
 (7, 4, 0, 1),
 (7, 1, 4, 0),
 (7, 1, 5, 0),
 (7, 2, 6, 0),
 (6, 3, 3, 0),
 (6, 4, 3, 0),
 (7, 1, 4, 1),
 (6, 1, 4, 1),
 (5, 2, 2, 1),
 (3, 1, 4, 1),
 (3, 2, 4, 1),
 (3, 3, 2, 1),
 (2, 4, 0, 1),
 [0, 1],
 [5, 1],
 (8, 1, 4, 0),
 (7, 2, 3, 0),
 (7, 3, 1, 0),
 

In [45]:
for j, game in enumerate(grouped_by_game):
    scores = collections.defaultdict(list)
    for play in game:
        scores[play['off']].append(play['offscore'])
        scores[play['def']].append(play['defscore'])

    for i in range(2, len(game)):
        for team in scores:
            try:
                if len(set((scores[team][i], scores[team][i-1], scores[team][i-2]))) == 3:
                    print(j, i,scores[team][i], scores[team][i-1], scores[team][i-2])
            except:
                pass

915 47 17 18 10
921 111 11 10 3
1401 2 17 10 3
2478 9 9 7 0


In [67]:
x = []
for game in grouped_by_game:
    c = 0
    for play in game:
        if play['time'] == 1800:
            c += 1
    x.append(c)